In [1]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
    


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
df= pd.read_csv('sample_dataset/train_hinglish_data_csv.csv', encoding = "ISO-8859-1", header=None)

In [24]:
df

,0,1,2
0,meta,4330,neutral
1,nen,Eng,NaN
2,á,O,NaN
3,vist,Eng,NaN
4,bolest,Eng,NaN
...,...,...,...
377031,t,Eng,NaN
377032,.,O,NaN
377033,co,Eng,NaN
377034,/,O,NaN


In [25]:
df[1] = df[1].replace('O', np.nan)
df = df.drop(axis=1, columns=[1])

In [26]:
df.head()

,0,2
0,meta,neutral
1,nen,NaN
2,á,NaN
3,vist,NaN
4,bolest,NaN


In [31]:
flag = 0

listRow = ''
currIndex = 0
for index, row in df.iterrows(): 
      if(row[0] == 'meta'):
        flag += 1
        if(flag ==2):           
            flag = 1
            listRow = ''
        currIndex = index
      else:      
          if(flag == 1):
            listRow+=(' '+str(row[0]))
          row[0] = np.nan
          df.iloc[currIndex][0] = listRow

    
#     if(row['name'] == 'aparna'):
#         row['name'] = df_test['name'].agg(lambda x: ' '.join(x.dropna()))
#         print(row['name'])


            

In [32]:
df = df.dropna()
df.tail()

,0,2
376889,@ anandk2012 @ RailMinIndia @ IRCTCofficial @...,negative
376914,so i couldn't get no one to cover me for my b...,positive
376943,RT @ theskindoctor13 @ NajimKhan07 Aap logo k...,positive
376973,@ PriyaVerma __ @ ashutosh83B Jay Jay Shree R...,positive
377004,@ kanpuriya _ Mujhe to bhayankar ho gaya hai ...,neutral


In [33]:
corpus = []
for index, row in df.iterrows(): 
    review = re.sub('[^a-zA-Z]', ' ', row[0])
    review = review.lower()
    review = review.split()
#     ps = PorterStemmer()
#     review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)
            

In [52]:
# Creating the Bag of Words model
# BOW model converts text in the form of numbers which can then be used in an algorithm for analysis.
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1000, lowercase=False)
X = cv.fit_transform(corpus).toarray()
y = df.iloc[:, 1].values

In [53]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [57]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y) 
onehotencoder = OneHotEncoder()
#reshape the 1-D country array to 2-D as fit_transform expects 2-D and finally fit the object 
y = onehotencoder.fit_transform(y.reshape(-1,1)).toarray()

In [58]:
y

array([[0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [59]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [60]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [61]:
y_train = np.array(y_train)

In [62]:
y_train

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [63]:
# Part 2 - Building the RNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [64]:
# Initialising the RNN
regressor = Sequential()

In [65]:
# Adding the first LSTM layer and some Dropout regularisation
#units is the number of neurons in first layer
#return_sequences = True means next layer will be added
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
#20% neurons will be dropped during forward and backward propogation
regressor.add(Dropout(0.2))

In [66]:
# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

In [67]:
# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

In [68]:
# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))


In [69]:
# Adding the output layer
regressor.add(Dense(units = 1))

In [70]:
# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [71]:
# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
137/336 [===========>..................] - ETA: 7:50 - loss: 0.2269

KeyboardInterrupt: 